In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json

In [2]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [3]:
# URL to find information on Coronavirus
url = "https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83"

url3 = "http://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus/Harris-County-COVID-19-Confirmed-Cases"

url4 = "https://www.khou.com/article/news/health/coronavirus/coronavirus-updates-live-blog-and-updates-for-march-26/285-51285105-6dd1-45fa-b169-a33310b5defc"

In [4]:
# Visiting the url 
browser.visit(url)

In [5]:
time.sleep(7)# Creating the Beautiful soup for URL
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# All the Texas counties
counties = soup.find('nav', attrs={'class':'feature-list'})

# getting all the spans with the information for each Texas county
rows = counties.find_all('span')




In [7]:
counter = 0

county = []
cases = []

for row in rows:
    try:
        print(rows[counter].text.strip().split('\n'))
        county.append(rows[counter].text.strip().split('\n')[0])
        cases.append(rows[counter].text.strip().split('\n')[1])
        counter = counter + 3
        
    except:
        print('failure')

['Dallas County', '367']
['Harris County', '203']
['Travis County', '137']
['Bexar County', '113']
['Tarrant County', '100']
['Collin County', '88']
['Fort Bend County', '86']
['Denton County', '83']
['Montgomery County', '41']
['Galveston County', '40']
['Brazoria County', '39']
['McLennan County', '33']
['Brazos County', '30']
['Williamson County', '27']
['Lubbock County', '23']
['El Paso County', '21']
['Smith County', '21']
['Bell County', '20']
['Nueces County', '18']
['Hays County', '13']
['Webb County', '13']
['Wichita County', '12']
['Hidalgo County', '11']
['Cameron County', '10']
['Ellis County', '10']
['Comal County', '9']
['Matagorda County', '9']
['Guadalupe County', '8']
['Jefferson County', '8']
['Castro County', '7']
['Johnson County', '6']
['Midland County', '6']
['Hockley County', '5']
['Gregg County', '4']
['Hardin County', '4']
['Kendall County', '4']
['Brown County', '3']
['De Witt County', '3']
['Medina County', '3']
['Rockwall County', '3']
['Victoria County', '3

In [8]:
df = pd.DataFrame(list(zip(county, cases)), columns =['Texas_County', 'COVID_19_Cases'])
df

,Texas_County,COVID_19_Cases
0,Dallas County,367
1,Harris County,203
2,Travis County,137
3,Bexar County,113
4,Tarrant County,100
...,...,...
100,Van Zandt County,1
101,Willacy County,1
102,Wilson County,1
103,Yoakum County,1


In [9]:
# Droping the word 'County'
df['Texas_County'] = df['Texas_County'].apply(lambda x : x.split(' County')[0])

In [10]:
# Revise? 
reported_cases = soup.find_all('text')[6].text
reported_cases

'1,731'

In [11]:
deaths = soup.find_all('text')[8].text
deaths

'23'

In [12]:
df_stats = pd.DataFrame({'Cases':[reported_cases], 'Deaths':[deaths]})
df_stats.to_csv('static/csv/stats.csv', index=False)

In [13]:
browser.visit(url3)

In [14]:
harris_table = browser.find_by_id('covidtable')

In [15]:
html_harris = browser.html
harris_soup = bs(html_harris, 'html.parser')

In [16]:
harris_data = []
table = harris_soup.find('table', attrs={'id':'covidtable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    harris_data.append([ele for ele in cols if ele]) # Get rid of empty values

In [17]:
columns = ['Patient', 'Sex', 'Age_Range', 'Location','Exposure', 'Status']


In [18]:
harris_data = harris_data[1:]

In [19]:
harris_df = pd.DataFrame(harris_data, columns=columns)

In [20]:
# harris_df['Sex'] = harris_df['Sex'].apply(lambda x : x.split()[2])

In [21]:
harris_df.to_csv('static/csv/harris_stats.csv', index=False, header=True)

In [22]:
harris_df.head()

,Patient,Sex,Age_Range,Location,Exposure,Status
0,1,Female,60 - 69,SW,History of Travel,Recovered
1,2,Male,60 - 69,SW,History of Travel,Recovered
2,3,Male,40 - 49,NW,Possible Exposure to Case,Active
3,4,Male,60 - 69,NW,History of Travel,Recovered
4,5,Male,40 - 49,NE,No Known Exposure,Active


In [23]:
All_Tex_Counties_df = pd.read_csv('static/csv/Texas_Counties.csv')
All_Tex_Counties_df.head()

,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"
3,-96.965687,30.310651,Lee,144,48287,2.213853,0.153990,"(-96.965687, 30.310651)"
4,-95.853568,32.211881,Henderson,108,48213,3.152909,0.235056,"(-95.853568, 32.211881)"


In [24]:
All_Tex_Counties = All_Tex_Counties_df['CNTY_NM'].tolist()

In [25]:
df_counties = df['Texas_County'].tolist()

In [26]:
Counties_with_Cases = []

for i in df_counties:
    
    if i in All_Tex_Counties:
        Counties_with_Cases.append(i)
        print(f'{i}---found')
        
    elif i == 'DeWitt':
        Counties_with_Cases.append('De Witt')
        print(f'{i}---found')
    else:
        print(f'Houston we have a problem: {i}')

Dallas---found
Harris---found
Travis---found
Bexar---found
Tarrant---found
Collin---found
Fort Bend---found
Denton---found
Montgomery---found
Galveston---found
Brazoria---found
McLennan---found
Brazos---found
Williamson---found
Lubbock---found
El Paso---found
Smith---found
Bell---found
Nueces---found
Hays---found
Webb---found
Wichita---found
Hidalgo---found
Cameron---found
Ellis---found
Comal---found
Matagorda---found
Guadalupe---found
Jefferson---found
Castro---found
Johnson---found
Midland---found
Hockley---found
Gregg---found
Hardin---found
Kendall---found
Brown---found
De Witt---found
Medina---found
Rockwall---found
Victoria---found
Wharton---found
Angelina---found
Atascosa---found
Bastrop---found
Bowie---found
Calhoun---found
Chambers---found
Deaf Smith---found
Eastland---found
Grayson---found
Grimes---found
Hood---found
Kaufman---found
Liberty---found
Llano---found
Milam---found
Parker---found
Rusk---found
Starr---found
Tom Green---found
Val Verde---found
Walker---found
Austin---

In [27]:
All_Tex_Counties_df = All_Tex_Counties_df.set_index('CNTY_NM')

In [28]:
Geo_df = All_Tex_Counties_df.loc[Counties_with_Cases]
Geo_df.head()

,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
CNTY_NM,,,,,,,
Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"


In [29]:
Geo_df = Geo_df.reset_index()
Geo_df = Geo_df.rename(columns={"CNTY_NM": "Texas_County"})
Geo_df

,Texas_County,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
1,Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
2,Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
4,Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
...,...,...,...,...,...,...,...,...
100,Van Zandt,-95.836486,32.563690,234,48467,2.402844,0.213777,"(-95.836486, 32.56369)"
101,Willacy,-97.643033,26.471329,245,48489,3.324169,0.155451,"(-97.643033, 26.471329)"
102,Wilson,-98.086210,29.173159,247,48493,1.917176,0.193944,"(-98.08621, 29.173159)"
103,Yoakum,-102.827879,33.172935,251,48501,1.791577,0.200360,"(-102.827879, 33.172935)"


In [30]:
result = pd.merge(df, Geo_df, on='Texas_County')
result = result.rename(columns={'X (Lat)':'long', 'Y (Long)':'lat'})

result.head()

,Texas_County,COVID_19_Cases,long,lat,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Dallas,367,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
1,Harris,203,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
2,Travis,137,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Bexar,113,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
4,Tarrant,100,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"


In [31]:
result = result[['Texas_County', 'COVID_19_Cases', 'lat', 'long']]
result

,Texas_County,COVID_19_Cases,lat,long
0,Dallas,367,32.766537,-96.777819
1,Harris,203,29.859671,-95.397821
2,Travis,137,30.334233,-97.781947
3,Bexar,113,29.448845,-98.519663
4,Tarrant,100,32.771852,-97.291165
...,...,...,...,...
100,Van Zandt,1,32.563690,-95.836486
101,Willacy,1,26.471329,-97.643033
102,Wilson,1,29.173159,-98.086210
103,Yoakum,1,33.172935,-102.827879


In [32]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lat],row[lon]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['Texas_County', 'COVID_19_Cases']
javascript_file = df_to_geojson(result, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [33]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)

In [34]:
browser.visit(url4)

In [35]:
headlines_harris = browser.html
headline_soup = bs(headlines_harris, 'html.parser')

In [36]:
texts=headline_soup.find('div', class_="text")

In [37]:
p_tags = texts.find_all('p')

In [38]:
paragraphs = []

In [39]:
for p in p_tags:
    
    try:
        strong = p.find('strong')
        print(strong.text + strong.next_sibling)
        paragraphs.append(strong.text + strong.next_sibling)
        
    except:
        print('fuc')
        continue

fuc
fuc
fuc
fuc
MARCH 26 8:59 p.m. — Two residents at a retirement community in The Woodlands have tested positive for COVID-19.
MARCH 26 8:59 p.m. — The city of Alvin has issued a stay-at-home order. The order mirrors what's already in place for Brazoria County, but city officials said it "supersedes all previous Emergency Declaration Orders issued" by the city council. The order also limits the number of people allowed in a funeral home to a maximum of 10 or fewer.
MARCH 26 8:54 p.m. — As Louisiana and New Orleans grapple with an increasing number of COVID-19 cases, local officials are criticizing the federal government for not providing more information ahead of Mardi Gras.
fuc
fuc
fuc
fuc
MARCH 26 7:20 p.m. — Scarface, the legendary Houston rapper, announced today that he’s tested positive for the coronavirus. He made the announcement on Willie D Live’s 
fuc
fuc
MARCH 26 5:58 p.m. — Chambers County has added a new amendment to an executive order permitting only essential businesses

In [40]:
paragraphs

['MARCH 26 8:59 p.m. —\xa0Two residents at a retirement community in The Woodlands have tested positive for COVID-19.',
 'MARCH 26 8:59 p.m. —\xa0The city of Alvin has issued a stay-at-home order. The order mirrors what\'s already in place for Brazoria County, but city officials said it "supersedes all previous Emergency Declaration Orders issued" by the city council. The order also limits the number of people allowed in a funeral home to a maximum of 10 or fewer.',
 'MARCH 26 8:54 p.m. —\xa0As Louisiana and New Orleans grapple with an increasing number of COVID-19 cases, local officials are criticizing the federal government for not providing more information ahead of Mardi Gras.',
 'MARCH 26 7:20 p.m. —\xa0Scarface, the legendary Houston rapper, announced today that he’s tested positive for the coronavirus. He made the announcement on Willie D Live’s ',
 'MARCH 26 5:58 p.m. —\xa0Chambers County has added a new amendment to an executive order permitting only essential businesses to re

In [41]:
mapped_para = list(map(lambda x: x.replace(u'\xa0', u' '), paragraphs))

In [42]:
mapped_para[:10]

['MARCH 26 8:59 p.m. — Two residents at a retirement community in The Woodlands have tested positive for COVID-19.',
 'MARCH 26 8:59 p.m. — The city of Alvin has issued a stay-at-home order. The order mirrors what\'s already in place for Brazoria County, but city officials said it "supersedes all previous Emergency Declaration Orders issued" by the city council. The order also limits the number of people allowed in a funeral home to a maximum of 10 or fewer.',
 'MARCH 26 8:54 p.m. — As Louisiana and New Orleans grapple with an increasing number of COVID-19 cases, local officials are criticizing the federal government for not providing more information ahead of Mardi Gras.',
 'MARCH 26 7:20 p.m. — Scarface, the legendary Houston rapper, announced today that he’s tested positive for the coronavirus. He made the announcement on Willie D Live’s ',
 'MARCH 26 5:58 p.m. — Chambers County has added a new amendment to an executive order permitting only essential businesses to remain open amid 

In [43]:
latest_headlines_df = pd.DataFrame({'Headline':mapped_para[:10]})

In [44]:
latest_headlines_df .to_csv('static/csv/headlines.csv', index=False, header=True)

In [45]:
browser.quit()